In [1]:
import cv2
import numpy as np



In [7]:
import cv2

def otsu_binarization(image):
  """
  Performs binarization of the input image using Otsu's thresholding method.

  Args:
      image: The input grayscale image.

  Returns:
      A binary image with values 1 (for white pixels) and values 0 (for black pixels).
  """
  _, binary_image = cv2.threshold(image, 0, 1, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
  return binary_image

def thinning(image):
  """
  Performs thinning of the binary image to remove extra data and allow for stroke verification.

  Args:
      image: The binary image.

  Returns:
      The thinned image.
  """
  kernel = np.ones((3, 3), np.uint8)
  skel = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)
  return skel

def core_zone_detection(image):
  """
  Detects the core zone of the word, which lies between the upper and lower baselines.

  Args:
      image: The binary image.

  Returns:
      The top and bottom coordinates of the core zone.
  """
  height, width = image.shape
  upper_baseline = 0
  lower_baseline = height - 1

  # Find upper baseline
  for row in range(height):
    if np.sum(image[row]) != np.sum(image[row - 1]):
      upper_baseline = row
      break

  # Find lower baseline (from bottom to top)
  for row in range(height - 2, -1, -1):
    if np.sum(image[row]) != np.sum(image[row + 1]):
      lower_baseline = row
      break

  return upper_baseline, lower_baseline

def segmentation(image, threshold):
  """
  Segments the core zone of the image into characters.

  Args:
      image: The binary image.
      threshold: A constant value to avoid over-segmentation.

  Returns:
      A list of character images.
  """
  candidate_segments = []
  height, width = image.shape
  for col in range(width):
    if np.sum(image[:, col]) == 0 or np.sum(image[:, col]) == 1:
      candidate_segments.append(col)

  # Apply threshold to avoid over-segmentation
  segmented_cols = [col for col in candidate_segments if np.sum(image[:, col]) <= threshold]

  character_images = []
  start = 0
  for col in segmented_cols:
    end = col
    character_image = image[:, start:end]
    character_images.append(character_image)
    start = end

  return character_images

def main():
  # Load the image
  image = cv2.imread("king.jpg", cv2.IMREAD_GRAYSCALE)

  # Check if image loaded successfully
  if image is None:
      print("Error: Could not load image!")
      return

  # Preprocessing
  binary_image = otsu_binarization(image)
  thinned_image = thinning(binary_image)

  # Core zone detection
  upper_baseline, lower_baseline = core_zone_detection(thinned_image)
  core_zone = thinned_image[upper_baseline:lower_baseline+1, :]

  # Segmentation
  segmented_characters = segmentation(core_zone, threshold=1)  # Adjust threshold as needed

  # Check if any characters were segmented
  if not segmented_characters:
      print("No characters found during segmentation!")
  else:
      # Display the segmented characters with debugging information
      for i, char in enumerate(segmented_characters):
          print(f"Character {i+1} shape: {char.shape}")
          cv2.imshow(f"Character {i+1}", char)
          cv2.waitKey(0)

if __name__ == "__main__":
  main()


Character 1 shape: (223, 0)


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


In [4]:
pip install OpenCV

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement OpenCV (from versions: none)
ERROR: No matching distribution found for OpenCV
